# Used Car Price Analysis
## Professional Certificate in Machine Learning and AI — UC Berkeley
### CRISP-DM Framework Application
---

## 1. Business Understanding

The goal of this project is to analyze a dataset of 426,000 used car listings to identify the key factors that influence used car prices. The findings will be delivered as actionable recommendations to a used car dealership to help them optimize their inventory decisions.

**Business Question:** What factors make a used car more or less expensive?

**Success Criteria:** Build a regression model that accurately predicts used car prices and identifies the most impactful features.

## 2. Data Understanding

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Load dataset
df = pd.read_csv('data/vehicles.csv')

print(f'Shape: {df.shape}')
print(f'\nData types:\n{df.dtypes}')
print(f'\nMissing values (%):\n{(df.isnull().sum() / len(df) * 100).sort_values(ascending=False).round(2)}')

In [ ]:
# Price distribution - raw vs clipped
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Price distribution (raw)', 'Price clipped at $100K'))

fig.add_trace(go.Histogram(x=df['price'], nbinsx=50, name='Raw price',
                           marker_color='steelblue'), row=1, col=1)
fig.add_trace(go.Histogram(x=df[df['price'] > 0]['price'].clip(upper=100000),
                           nbinsx=50, name='Clipped price',
                           marker_color='darkorange'), row=1, col=2)

fig.update_layout(title='Price Distribution', height=400, showlegend=False)
fig.show()

print(df['price'].describe())
print(f'\nCars with price = 0: {(df["price"] == 0).sum()}')
print(f'Cars with price > 100,000: {(df["price"] > 100000).sum()}')

## 3. Data Preparation

In [ ]:
# Step 1: Filter price outliers
df_clean = df[(df['price'] >= 500) & (df['price'] <= 100000)]
print(f'Rows after price filter: {len(df_clean)}')

# Step 2: Drop low-value columns
df_clean = df_clean.drop(columns=['id', 'VIN', 'size', 'region', 'model'])

# Step 3: Drop rows with nulls in key columns
df_clean = df_clean.dropna(subset=['year', 'odometer', 'fuel', 'transmission', 'title_status'])

# Step 4: Fill remaining nulls with 'unknown'
cols_fill = ['manufacturer', 'condition', 'cylinders', 'drive', 'paint_color', 'type']
df_clean[cols_fill] = df_clean[cols_fill].fillna('unknown')

# Step 5: Filter year and odometer outliers
df_clean = df_clean[(df_clean['year'] >= 1990) & (df_clean['year'] <= 2022)]
df_clean = df_clean[df_clean['odometer'] <= 300000]
df_clean = df_clean[df_clean['odometer'] > 1000]

print(f'Final rows: {len(df_clean)}')
print(f'\nNull values remaining:\n{df_clean.isnull().sum()}')
print(f'\nFinal dataset summary:\n{df_clean.describe()}')

## 4. Exploratory Data Analysis (EDA)

In [ ]:
# Correlation heatmap
corr_matrix = df_clean[['price', 'year', 'odometer']].corr()

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values.round(2),
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.columns.tolist(),
    colorscale='RdBu', showscale=True, reversescale=True)

fig.update_layout(title='Numerical Attributes Correlation', height=400, width=500)
fig.show()

In [ ]:
# Median price by year
median_by_year = df_clean.groupby('year')['price'].median().reset_index()

fig = go.Figure(go.Scatter(
    x=median_by_year['year'], y=median_by_year['price'],
    mode='lines+markers', line=dict(color='steelblue', width=2),
    hovertemplate='Year: %{x}<br>Median price: $%{y:,.0f}<extra></extra>'
))
fig.update_layout(title='Median Price by Year', xaxis_title='Year',
                  yaxis_title='Median Price ($)', height=400)
fig.show()

In [ ]:
# Price vs Odometer
sample = df_clean.sample(5000, random_state=42)

fig = go.Figure(go.Scatter(
    x=sample['odometer'], y=sample['price'], mode='markers',
    marker=dict(color='steelblue', opacity=0.3, size=5),
    hovertemplate='Odometer: %{x:,.0f} miles<br>Price: $%{y:,.0f}<extra></extra>'
))
fig.update_layout(title='Price vs Odometer', xaxis_title='Odometer (miles)',
                  yaxis_title='Price ($)', height=500)
fig.show()

In [ ]:
# Top 10 manufacturers by median price
top_manufacturers = (df_clean.groupby('manufacturer')['price']
                     .median().sort_values(ascending=False).head(10).reset_index())

fig = go.Figure(go.Bar(
    x=top_manufacturers['manufacturer'], y=top_manufacturers['price'],
    marker_color='steelblue',
    hovertemplate='Manufacturer: %{x}<br>Median price: $%{y:,.0f}<extra></extra>'
))
fig.update_layout(title='Top 10 Manufacturers by Median Price',
                  xaxis_title='Manufacturer', yaxis_title='Median Price ($)',
                  height=400, xaxis_tickangle=-45)
fig.show()

In [ ]:
# Price by condition
order = ['new', 'like new', 'excellent', 'good', 'fair', 'salvage', 'unknown']
fig = go.Figure()
for condition in order:
    fig.add_trace(go.Box(
        y=df_clean[df_clean['condition'] == condition]['price'], name=condition,
        hovertemplate='Condition: ' + condition + '<br>Price: $%{y:,.0f}<extra></extra>'
    ))
fig.update_layout(title='Price per Condition', xaxis_title='Condition',
                  yaxis_title='Price ($)', height=450, showlegend=False)
fig.show()

In [ ]:
# Price by fuel type
fig = go.Figure()
for fuel in df_clean['fuel'].unique():
    fig.add_trace(go.Box(
        y=df_clean[df_clean['fuel'] == fuel]['price'], name=fuel,
        hovertemplate='Fuel: ' + fuel + '<br>Price: $%{y:,.0f}<extra></extra>'
    ))
fig.update_layout(title='Price per Fuel Type', height=450, showlegend=False)
fig.show()

In [ ]:
# Price by drive type
fig = go.Figure()
for drive in df_clean['drive'].unique():
    fig.add_trace(go.Box(
        y=df_clean[df_clean['drive'] == drive]['price'], name=drive,
        hovertemplate='Drive: ' + drive + '<br>Price: $%{y:,.0f}<extra></extra>'
    ))
fig.update_layout(title='Price per Drive Type', height=450, showlegend=False)
fig.show()

In [ ]:
# Price by body type
fig = go.Figure()
for body in df_clean['type'].unique():
    fig.add_trace(go.Box(
        y=df_clean[df_clean['type'] == body]['price'], name=body,
        hovertemplate='Type: ' + body + '<br>Price: $%{y:,.0f}<extra></extra>'
    ))
fig.update_layout(title='Price per Body Type', height=450,
                  xaxis_tickangle=-45, showlegend=False)
fig.show()

In [ ]:
# Top 15 manufacturers by volume
top_volume = df_clean['manufacturer'].value_counts().head(15).reset_index()
top_volume.columns = ['manufacturer', 'count']

fig = go.Figure(go.Bar(
    x=top_volume['manufacturer'], y=top_volume['count'],
    marker_color='steelblue',
    hovertemplate='Manufacturer: %{x}<br>Count: %{y:,.0f}<extra></extra>'
))
fig.update_layout(title='Top 15 Manufacturers by Volume',
                  xaxis_title='Manufacturer', yaxis_title='Number of Cars',
                  height=400, xaxis_tickangle=-45)
fig.show()

## 5. Modeling

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Define feature groups
numeric_features = ['year', 'odometer']
ordinal_features = ['condition']
nominal_features = ['manufacturer', 'cylinders', 'fuel', 'title_status',
                    'transmission', 'drive', 'type', 'paint_color', 'state']

ordinal_categories = [['unknown', 'salvage', 'fair', 'good', 'excellent', 'like new', 'new']]

# Preprocessor with scaling
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('ord', OrdinalEncoder(categories=ordinal_categories), ordinal_features),
    ('nom', OneHotEncoder(handle_unknown='ignore'), nominal_features)
])

# Features and target
X = df_clean[numeric_features + ordinal_features + nominal_features]
y = df_clean['price']

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluation function
def evaluate_cv(name, pipeline, X, y, cv):
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_squared_error')
    mse_mean = -scores.mean()
    mse_std = scores.std()
    print(f'\n{name}')
    print(f'  MSE:  {mse_mean:,.0f}')
    print(f'  RMSE: ${np.sqrt(mse_mean):,.0f} (+/- ${np.sqrt(mse_std):,.0f})')

# Linear Regression baseline
pipe_lr = Pipeline([('preprocessor', preprocessor), ('model', LinearRegression())])
evaluate_cv('Linear Regression', pipe_lr, X, y, kf)

# Ridge - different alphas
for alpha in [0.1, 1.0, 10.0, 100.0]:
    pipe = Pipeline([('preprocessor', preprocessor), ('model', Ridge(alpha=alpha))])
    evaluate_cv(f'Ridge alpha={alpha}', pipe, X, y, kf)

# Lasso - different alphas
for alpha in [0.1, 1.0, 10.0, 100.0]:
    pipe = Pipeline([('preprocessor', preprocessor), ('model', Lasso(alpha=alpha, max_iter=5000))])
    evaluate_cv(f'Lasso alpha={alpha}', pipe, X, y, kf)

## 6. Evaluation

### Model Performance Summary

| Model | MSE | RMSE | Stability (±) |
|-------|-----|------|---------------|
| Linear Regression | 73,447,251 | $8,570 | ±$873 |
| Ridge alpha=0.1 | 60,794,911 | $7,797 | ±$821 |
| **Ridge alpha=1.0** | **60,792,907** | **$7,797** | **±$820** |
| Ridge alpha=10.0 | 60,806,865 | $7,798 | ±$815 |
| Ridge alpha=100.0 | 61,008,273 | $7,811 | ±$817 |
| Lasso alpha=0.1 | 60,795,012 | $7,797 | ±$821 |
| Lasso alpha=1.0 | 60,822,451 | $7,799 | ±$818 |
| Lasso alpha=10.0 | 62,002,429 | $7,874 | ±$838 |
| Lasso alpha=100.0 | 69,218,468 | $8,320 | ±$905 |

**Selected model: Ridge Regression (alpha=1.0)**
- Reduces MSE by 17% vs Linear Regression baseline
- Best stability across cross-validation folds
- No convergence issues unlike Lasso at low alphas

In [ ]:
# Train final model and extract coefficients
best_model = Pipeline([('preprocessor', preprocessor), ('model', Ridge(alpha=1.0))])
best_model.fit(X, y)

# Extract feature names
ohe_features = (best_model.named_steps['preprocessor']
                .named_transformers_['nom']
                .get_feature_names_out(nominal_features))

all_features = numeric_features + ordinal_features + list(ohe_features)

# Build coefficients dataframe
coefficients = pd.DataFrame({
    'feature': all_features,
    'coefficient': best_model.named_steps['model'].coef_
}).sort_values('coefficient', key=abs, ascending=False)

print('Top 15 features that INCREASE price:')
print(coefficients[coefficients['coefficient'] > 0].head(15)[['feature', 'coefficient']].to_string())

print('\nTop 15 features that DECREASE price:')
print(coefficients[coefficients['coefficient'] < 0].head(15)[['feature', 'coefficient']].to_string())

## 7. Deployment — Recommendations to the Client

---

### Executive Summary

Using a dataset of 351,000+ used car listings and a Ridge Regression model with a cross-validated RMSE of **$7,797**, we identified the key factors that drive used car prices in the US market. The findings below provide actionable recommendations to optimize inventory decisions.

---

### Key Findings

**1. Vehicle Age is the Most Powerful Price Driver**  
Year is the single most impactful numeric feature (+$5,496 per standardized unit). Dealers should prioritize acquiring vehicles from **2015 onwards** to maximize resale margins.

**2. Brand Matters — Selectively**  
Premium brands command significant premiums: Ferrari (+$50,631), Tesla (+$15,671), Porsche (+$9,847). On the other hand, Fiat (-$10,271), Mitsubishi (-$7,702) and Kia (-$6,510) are discounted by the market. Only acquire discounted brands if the purchase price reflects the market discount.

**3. Engine Size and Fuel Type Are Critical**  
12-cylinder engines add +$13,507. Diesel adds +$12,250. 3-cylinder engines lose -$6,065. Stock with larger engines and diesel vehicles commands higher margins.

**4. Mileage Has a Clear Negative Impact**  
Odometer is the second most impactful numeric feature (-$4,753). Focus inventory on vehicles **under 75,000 miles** when possible.

**5. Vehicle Type: Pickups and Off-Road Win**  
Pickups (+$2,886), convertibles (+$3,157) and off-road (+$3,875) command clear premiums. The American market values trucks and utility vehicles consistently.

**6. Clean Title is Non-Negotiable**  
A clean title adds +$3,362. Salvage or rebuilt titles significantly reduce resale value.

**7. Electric Vehicles — Proceed with Caution**  
EVs show a negative coefficient (-$6,005) in this dataset, reflecting older models with limited range. Only Tesla and recent premium EVs are exceptions.

---

### Recommendations

1. **Prioritize model years 2015+** — the single biggest lever on price
2. **Focus on low mileage stock (under 75,000 miles)** — clear and consistent price impact
3. **Invest in pickups, trucks and off-road vehicles** — strong demand and price premium
4. **Acquire premium brands selectively** — Ferrari, Porsche, Tesla command strong premiums
5. **Avoid high-volume low-margin brands** — Fiat, Mitsubishi, Kia unless acquisition price reflects market discount
6. **Always verify clean title** — a lien or salvage title significantly erodes value
7. **Prioritize diesel and large engine vehicles** — strong price premium especially for work trucks